In [1]:

from typing import Sequence

import torch
import numpy as np
from models import MLPModel, FancyMLPModel
from agents import Agent
from collectors import Memory, CrowdCollector
from environments import UnityCrowdEnv, UnitySimpleCrowdEnv
from policy_optimization import CrowdPPOptimizer
from trainers import PPOCrowdTrainer

from tqdm import tqdm, trange
from mlagents_envs.environment import UnityEnvironment

from utils import transpose_batch, concat_batches, concat_crowd_batch, tanh_norm, atanh_unnorm

from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

import seaborn as sns
import matplotlib.pyplot as plt

from utils import discount_rewards_to_go, get_episode_lens

In [2]:

env = UnitySimpleCrowdEnv(file_name=None)




env.engine_channel.set_configuration_parameters(time_scale=100)

This means that some features may not work unless you upgrade the package with the lower version.Please find the versions that work best together from our release page.
https://github.com/Unity-Technologies/ml-agents/releases


In [3]:
action_range = (
    torch.tensor([-.3, -1.]),
    torch.tensor([ 1.,  1.])
)

In [4]:
model = FancyMLPModel({
    "input_size": 94,
})

agent = Agent(model, action_range=action_range)

In [3]:
obs = env.reset()

In [8]:
obs['Person1?team=0&id=1'][:20]

array([1000000., 1000000., 1000000., 1000000., 1000000., 1000000.,
       1000000., 1000000., 1000000., 1000000., 1000000., 1000000.,
       1000000., 1000000., 1000000., 1000000., 1000000., 1000000.,
       1000000., 1000000.], dtype=float32)

In [5]:
collector = CrowdCollector(agent, env)

In [6]:
batch, metrics = collector.collect_data(100, disable_tqdm=False)

100%|██████████| 100/100 [00:01<00:00, 54.05it/s]


In [13]:
evals = agent.evaluate_actions(concat_crowd_batch(batch))

In [14]:
evals

(tensor([-2.0285, -3.4034, -1.9741,  ..., -1.0643, -0.5024, -1.6161],
        grad_fn=<SumBackward1>),
 tensor([-0.6255, -1.1358, -0.8687,  ...,  0.1529, -0.0524, -0.0520],
        grad_fn=<ViewBackward>),
 tensor([2.5236, 2.5276, 2.8892,  ..., 1.3751, 1.3518, 1.3419],
        grad_fn=<SumBackward1>))

In [7]:
import pdb; pdb.pm()

> /Users/redtachyon/projects/CrowdAI/training/collectors.py(200)collect_data()
    198 
    199             obs_array, agent_keys = pack(obs_dict)
--> 200             obs_tensor = torch.tensor(obs_array)
    201 
    202             # Centralize the action computation for better parallelization

array([array([ 0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0

In [9]:
env.close()